In [12]:
%pip install nltk
%pip install underthesea

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import re
import nltk
from underthesea import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

file_path = "./dataset_vi/Amazon_Product_Review_full_vi.csv"
df = pd.read_csv(file_path)


# In ra tên cột thực tế để kiểm tra
print("Tên cột thực tế:", df.columns.tolist())

# Chuẩn hóa tên cột (loại bỏ khoảng trắng thừa và BOM nếu có)
df.columns = df.columns.str.strip()
df.columns = [col.replace('\ufeff', '') for col in df.columns]

# Nếu vẫn chưa có cột, in thử 5 dòng đầu để kiểm tra
if not set(['review_body', 'review_headline']).issubset(df.columns):
    print("5 dòng đầu của file:")
    print(df.head())
    # Nếu không có header, đặt lại tên cột (sửa lại cho đúng số cột)
    # df.columns = ['review_body', 'review_headline', 'sentiment']

if set(['review_body', 'review_headline']).issubset(df.columns):
    df = df.dropna(subset=['review_body', 'review_headline'])
    df = df[~df['review_body'].str.strip().eq('')]
    df = df[~df['review_headline'].str.strip().eq('')]

    if 'sentiment' in df.columns:
        df['sentiment'] = pd.to_numeric(df['sentiment'], errors='coerce')
        df = df.dropna(subset=['sentiment'])
        min_count = df['sentiment'].value_counts().min()
        df = df.groupby('sentiment').sample(n=min_count, random_state=42)
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)
        print('Số lượng mẫu mỗi lớp sau khi cân bằng:', df['sentiment'].value_counts().to_dict())

    df = df.drop_duplicates()
    df = df.dropna()

    vi_stopwords = set([
        "và", "là", "của", "có", "cho", "một", "các", "những", "được", "trong", "khi", "đã", "với", "rằng", "thì", "lại", "này", "ở", "ra", "từ", "sẽ", "về", "như", "đó", "nên", "đi", "nữa", "cũng", "rất", "để", "nhưng", "bị", "theo", "vẫn", "vì", "nào", "đến", "này", "tôi", "anh", "em", "chị", "bạn", "chúng", "ta", "họ", "ai", "gì", "đâu", "đây", "đấy", "ấy", "vậy", "thế", "này", "kia", "kìa"
    ])

    def clean_text_vi(text):
        if not isinstance(text, str):
            return ''
        text = re.sub(r'<[^>]+>', '', text)
        text = text.lower()
        text = re.sub(r'[^a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ\s]', '', text)
        tokens = word_tokenize(text, format="text").split()
        tokens = [w for w in tokens if w not in vi_stopwords and len(w) > 1]
        return ' '.join(tokens)

    df['review_body'] = df['review_body'].astype(str).apply(clean_text_vi)
    df['review_headline'] = df['review_headline'].astype(str).apply(clean_text_vi)

    df = df[~df['review_body'].str.strip().eq('')]
    df = df[~df['review_headline'].str.strip().eq('')]

    print("Số dòng sau khi làm sạch:", len(df))
    cleaned_file_path = "./dataset_vi/Amazon_Product_Review_full_vi_cleaned.csv"
    df.to_csv(cleaned_file_path, index=False)

    print("Làm sạch hoàn tất. File được lưu tại:", cleaned_file_path)
else:
    print("File của bạn không có đủ cột 'review_body' và 'review_headline'. Hãy kiểm tra lại file CSV!")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NCPC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NCPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\NCPC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Tên cột thực tế: ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date', 'sentiment']
Số lượng mẫu mỗi lớp sau khi cân bằng: {0: 5079, 1: 5079}
Số dòng sau khi làm sạch: 10108
Làm sạch hoàn tất. File được lưu tại: ./dataset_vi/Amazon_Product_Review_full_vi_cleaned.csv
